In [2]:
!pip install pyspark

In [3]:
import pandas as pd
import sqlite3
from pyspark.sql import SparkSession

In [4]:
# Load the dataset from the provided URL
url = "https://data.sfgov.org/api/views/wr8u-xric/rows.csv?accessType=DOWNLOAD"
df = pd.read_csv(url)

# Display the first few rows
df.head()

<ipython-input-4-72dba179d7b2>:3: DtypeWarning: Columns (10,12,13,20,29,38,45,46,55,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


,Incident Number,Exposure Number,ID,Address,Incident Date,Call Number,Alarm DtTm,Arrival DtTm,Close DtTm,City,...,Automatic Extinguishing System Present,Automatic Extinguishing Sytem Type,Automatic Extinguishing Sytem Perfomance,Automatic Extinguishing Sytem Failure Reason,Number of Sprinkler Heads Operating,Supervisor District,neighborhood_district,point,data_as_of,data_loaded_at
0,20010694,0,200106940,847 FAIRFAX AV 301,2020/01/25,200252338,2020/01/25 03:45:23 PM,2020/01/25 03:51:00 PM,2020/01/25 04:03:28 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,POINT (-122.379762 37.735188),2020/01/25 04:03:28 PM,2025/02/19 02:24:48 AM
1,20010701,0,200107010,4657 MISSION STREET,2020/01/25,200252384,2020/01/25 03:58:31 PM,2020/01/25 04:03:04 PM,2020/01/25 04:40:11 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,11.0,Excelsior,POINT (-122.43528 37.723915),2020/01/25 04:40:11 PM,2025/02/19 02:24:48 AM
2,20010702,0,200107020,914 POINT LOBOS AVENUE,2020/01/25,200252385,2020/01/25 03:58:31 PM,2020/01/25 04:05:47 PM,2020/01/25 04:40:33 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,1.0,Outer Richmond,POINT (-122.513698 37.779944),2020/01/25 04:40:33 PM,2025/02/19 02:24:48 AM
3,20010710,0,200107100,150 FONT,2020/01/25,200252473,2020/01/25 04:20:31 PM,2020/01/25 04:23:48 PM,2020/01/25 04:38:24 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,7.0,Lakeshore,POINT (-122.473678 37.715515),2020/01/25 04:38:24 PM,2025/02/19 02:24:48 AM
4,20010714,0,200107140,292 14TH STREET,2020/01/25,200252546,2020/01/25 04:35:29 PM,2020/01/25 04:39:49 PM,2020/01/25 04:47:17 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,9.0,Mission,POINT (-122.419801 37.768352),2020/01/25 04:47:17 PM,2025/02/19 02:24:48 AM


In [5]:
df_selected = df[['Incident Number', 'Address', 'Incident Date', 'Primary Situation']]
df_selected.head()

,Incident Number,Address,Incident Date,Primary Situation
0,20010694,847 FAIRFAX AV 301,2020/01/25,"700 False alarm or false call, other"
1,20010701,4657 MISSION STREET,2020/01/25,"500 Service Call, other"
2,20010702,914 POINT LOBOS AVENUE,2020/01/25,"350 Extrication, rescue, other"
3,20010710,150 FONT,2020/01/25,"743 Smoke detector activation, no fire - unint..."
4,20010714,292 14TH STREET,2020/01/25,"712 Direct tie to FD, malicious false alarm"


In [6]:
df_2023 = df[df['Incident Date'].str.contains("2023", na=False)]
df_2023.head()

,Incident Number,Exposure Number,ID,Address,Incident Date,Call Number,Alarm DtTm,Arrival DtTm,Close DtTm,City,...,Automatic Extinguishing System Present,Automatic Extinguishing Sytem Type,Automatic Extinguishing Sytem Perfomance,Automatic Extinguishing Sytem Failure Reason,Number of Sprinkler Heads Operating,Supervisor District,neighborhood_district,point,data_as_of,data_loaded_at
361022,23000004,0,230000040,125-A PRECITA AVENUE,2023/01/01,230010028,2023/01/01 12:07:44 AM,2023/01/01 12:10:42 AM,2023/01/01 12:28:03 AM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,9.0,Bernal Heights,POINT (-122.417759 37.746973),2023/01/01 12:28:03 AM,2025/02/19 02:24:48 AM
361023,23000005,0,230000050,1425 FILLMORE STREET,2023/01/01,230010051,2023/01/01 12:14:02 AM,2023/01/01 12:16:29 AM,2023/01/01 12:30:21 AM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,5.0,Western Addition,POINT (-122.432655 37.782604),2023/01/01 12:30:21 AM,2025/02/19 02:24:48 AM
361024,23000008,0,230000080,1188 FOLSOM STREET,2023/01/01,230010058,2023/01/01 12:17:05 AM,2023/01/01 12:21:29 AM,2023/01/01 12:28:26 AM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,6.0,South of Market,POINT (-122.409874 37.775239),2023/01/01 12:28:26 AM,2025/02/19 02:24:48 AM
361025,23000011,0,230000110,GEARY STREET,2023/01/01,230010072,2023/01/01 12:20:18 AM,2023/01/01 12:23:53 AM,2023/01/01 12:26:00 AM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,5.0,Tenderloin,POINT (-122.412344 37.786834),2023/01/01 12:26:00 AM,2025/02/19 02:24:48 AM
361026,23000019,0,230000190,388 BEALE STREET,2023/01/01,230010118,2023/01/01 12:32:48 AM,2023/01/01 12:39:00 AM,2023/01/01 12:48:55 AM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,6.0,Financial District/South Beach,POINT (-122.39145 37.787572),2023/01/01 12:48:55 AM,2025/02/19 02:24:48 AM


In [7]:
df_cleaned = df.dropna(subset=['Address'])
df_cleaned.head()

,Incident Number,Exposure Number,ID,Address,Incident Date,Call Number,Alarm DtTm,Arrival DtTm,Close DtTm,City,...,Automatic Extinguishing System Present,Automatic Extinguishing Sytem Type,Automatic Extinguishing Sytem Perfomance,Automatic Extinguishing Sytem Failure Reason,Number of Sprinkler Heads Operating,Supervisor District,neighborhood_district,point,data_as_of,data_loaded_at
0,20010694,0,200106940,847 FAIRFAX AV 301,2020/01/25,200252338,2020/01/25 03:45:23 PM,2020/01/25 03:51:00 PM,2020/01/25 04:03:28 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,POINT (-122.379762 37.735188),2020/01/25 04:03:28 PM,2025/02/19 02:24:48 AM
1,20010701,0,200107010,4657 MISSION STREET,2020/01/25,200252384,2020/01/25 03:58:31 PM,2020/01/25 04:03:04 PM,2020/01/25 04:40:11 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,11.0,Excelsior,POINT (-122.43528 37.723915),2020/01/25 04:40:11 PM,2025/02/19 02:24:48 AM
2,20010702,0,200107020,914 POINT LOBOS AVENUE,2020/01/25,200252385,2020/01/25 03:58:31 PM,2020/01/25 04:05:47 PM,2020/01/25 04:40:33 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,1.0,Outer Richmond,POINT (-122.513698 37.779944),2020/01/25 04:40:33 PM,2025/02/19 02:24:48 AM
3,20010710,0,200107100,150 FONT,2020/01/25,200252473,2020/01/25 04:20:31 PM,2020/01/25 04:23:48 PM,2020/01/25 04:38:24 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,7.0,Lakeshore,POINT (-122.473678 37.715515),2020/01/25 04:38:24 PM,2025/02/19 02:24:48 AM
4,20010714,0,200107140,292 14TH STREET,2020/01/25,200252546,2020/01/25 04:35:29 PM,2020/01/25 04:39:49 PM,2020/01/25 04:47:17 PM,San Francisco,...,NaN,NaN,NaN,NaN,NaN,9.0,Mission,POINT (-122.419801 37.768352),2020/01/25 04:47:17 PM,2025/02/19 02:24:48 AM


In [8]:
# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
df.to_sql('fire_incidents', conn, index=False, if_exists='replace')

# SQL Query: Get Top 5 Most Frequent Primary Situations
query = """
SELECT "Primary Situation", COUNT(*) as count
FROM fire_incidents
GROUP BY "Primary Situation"
ORDER BY count DESC
LIMIT 5
"""
top_situations = pd.read_sql(query, conn)
top_situations

,Primary Situation,count
0,"711 - Municipal alarm system, Street Box False",53324
1,"700 False alarm or false call, other",41301
2,"700 - False alarm or false call, other",22254
3,"745 Alarm system activation, no fire - uninten...",20966
4,745 - Alarm system sounded/no fire-accidental,19571


In [9]:
# Initialize Spark Session
spark = SparkSession.builder.appName("FireIncidents").getOrCreate()

In [11]:
from pyspark.sql.types import *

# Define the schema explicitly
schema = StructType([
    StructField("Incident Number", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("Incident Date", StringType(), True),
    StructField("Primary Situation", StringType(), True)
])

# Convert Pandas DataFrame to PySpark DataFrame with schema
df_spark = spark.createDataFrame(df_selected.astype(str), schema=schema)

# Show the first few rows
df_spark.show(5)

+---------------+--------------------+-------------+--------------------+
|Incident Number|             Address|Incident Date|   Primary Situation|
+---------------+--------------------+-------------+--------------------+
|       20010694|  847 FAIRFAX AV 301|   2020/01/25|700 False alarm o...|
|       20010701| 4657 MISSION STREET|   2020/01/25|500 Service Call,...|
|       20010702|914 POINT LOBOS A...|   2020/01/25|350 Extrication, ...|
|       20010710|            150 FONT|   2020/01/25|743 Smoke detecto...|
|       20010714|     292 14TH STREET|   2020/01/25|712 Direct tie to...|
+---------------+--------------------+-------------+--------------------+
only showing top 5 rows



In [12]:
df_filtered = df_spark.filter(df_spark["Incident Date"].contains("2023"))
df_filtered.show(5)

+---------------+--------------------+-------------+--------------------+
|Incident Number|             Address|Incident Date|   Primary Situation|
+---------------+--------------------+-------------+--------------------+
|       23000004|125-A PRECITA AVENUE|   2023/01/01|113 Cooking fire,...|
|       23000005|1425 FILLMORE STREET|   2023/01/01|745 Alarm system ...|
|       23000008|  1188 FOLSOM STREET|   2023/01/01|733 Smoke detecto...|
|       23000011|        GEARY STREET|   2023/01/01|700 False alarm o...|
|       23000019|    388 BEALE STREET|   2023/01/01|700 False alarm o...|
+---------------+--------------------+-------------+--------------------+
only showing top 5 rows



In [13]:
df_grouped = df_spark.groupBy("Primary Situation").count().orderBy("count", ascending=False)
df_grouped.show(5)

+--------------------+-----+
|   Primary Situation|count|
+--------------------+-----+
|711 - Municipal a...|53324|
|700 False alarm o...|41301|
|700 - False alarm...|22254|
|745 Alarm system ...|20966|
|745 - Alarm syste...|19571|
+--------------------+-----+
only showing top 5 rows



In [14]:
# Save cleaned dataset to CSV
df_cleaned.to_csv("cleaned_fire_incidents.csv", index=False)

# Save cleaned dataset to JSON
df_cleaned.to_json("cleaned_fire_incidents.json", orient="records")

# Save cleaned dataset to TXT (tab-separated)
df_cleaned.to_csv("cleaned_fire_incidents.txt", sep="\t", index=False)

print("Files saved: cleaned_fire_incidents.csv, cleaned_fire_incidents.json, cleaned_fire_incidents.txt")

Files saved: cleaned_fire_incidents.csv, cleaned_fire_incidents.json, cleaned_fire_incidents.txt
